[![Open in SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ninjaneural/sagemaker/blob/main/notebook/webui.ipynb)

In [ ]:
# 실행변수
HomePath = '/home/studio-lab-user'
Workspace = 'ui'


In [ ]:
# 설치
HomePath = '/home/studio-lab-user'

#**초기모델(checkpoint) URL**
Checkpoint_Url = 'https://civitai.com/api/download/models/94640?type=Model&format=SafeTensor&size=pruned&fp=fp16'
#**초기모델 파일명**
Checkpoint_Filename = 'majicMIX_realistic_v6.safetensors'

#**ControlNet을 사용하지 않으면 체크를 해제해주세요**
ControlNet = False

#**After Detailer(adetailer)를 사용하지 않으면 체크를 해제해주세요**
ADetailer = False

#**Dectect Detailer(ddetailer)를 사용하지 않으면 체크를 해제해주세요**
DDetailer = False

#**Segment Anything을 사용하지 않으면 체크를 해제해주세요**
SegmentAnything = False

Workspace = 'ui'
TagVersion = 'v1.6.0'

!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -q xformers==0.0.20 triton==2.0.0 -U
!pip install httpx==0.24.1

# install util
!conda install -q -y aria2
!conda install glib=2.51.0 -y

%cd {HomePath}
!wget https://raw.githubusercontent.com/ninjaneural/webui/master/misc/install_sagemaker.sh -O ./install.sh
!bash install.sh {Workspace} {TagVersion} {ControlNet} {DDetailer} {ADetailer} {SegmentAnything}

%cd {HomePath}/{Workspace}
!wget https://raw.githubusercontent.com/ninjaneural/webui/master/misc/config_api.json -O ./config.json

# checkpoint select
!sed -i -e 's/"sd_model_checkpoint": "",/"sd_model_checkpoint": "{Checkpoint_Filename}",/g' ./config.json

# 추가 익스텐션
# git clone https://github.com/KohakuBlueleaf/a1111-sd-webui-haku-img ./extensions/a1111-sd-webui-haku-img

# checkpoint 
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{Checkpoint_Url}" -d ./models/Stable-diffusion -o {Checkpoint_Filename}

# upscaler
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d ./models/ESRGAN -o 4x-UltraSharp.pth

# vae
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d ./models/Stable-diffusion -o vae-ft-mse-840000-ema-pruned.vae.pt

# lora
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/87153?type=Model&format=SafeTensor" -d ./models/Lora -o more_details.safetensors

# embeddings
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.pt -d ./embeddings -o EasyNegative.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/AsciiP/badhandv4/resolve/main/badhandv4.pt -d ./embeddings -o badhandv4.pt

!conda clean --all -y
!rm -rf {HomePath}/.cache

print("설치 완료! 시작을 실행해주세요")


In [ ]:
# controlnet model 다운로드
if 'HomePath' in locals():

    %cd {HomePath}/{Workspace}

    # 모델삭제
    !rm -r ./extensions/controlnet/models/*

    # basic
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -d ./extensions/controlnet/models -o control_v11p_sd15_openpose_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -d ./extensions/controlnet/models -o control_v11p_sd15_lineart_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -d ./extensions/controlnet/models -o control_v11p_sd15_softedge_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -d ./extensions/controlnet/models -o control_v11f1p_sd15_depth_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -d ./extensions/controlnet/models -o control_v11p_sd15_canny_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -d ./extensions/controlnet/models -o control_v11e_sd15_ip2p_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors -d ./extensions/controlnet/models -o control_v11f1e_sd15_tile_fp16.safetensors

    # !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -d ./extensions/controlnet/models -o control_v11p_sd15_scribble_fp16.safetensors
    # !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -d ./extensions/controlnet/models -o control_v11p_sd15_seg_fp16.safetensors
    # !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -d ./extensions/controlnet/models -o control_v11p_sd15_inpaint_fp16.safetensors
    # !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -d ./extensions/controlnet/models -o control_v11p_sd15s2_lineart_anime_fp16.safetensors

    # #temporalnet
    # !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/CiaraRowles/TemporalNet/resolve/main/diff_control_sd15_temporalnet_fp16.safetensors -d ./extensions/controlnet/models -o diff_control_sd15_temporalnet_fp16.safetensors

    # #ip-adapter
    # !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.bin -d ./extensions/controlnet/models -o ip-adapter_sd15.pth
    # !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.bin -d ./extensions/controlnet/models -o ip-adapter-plus_sd15.pth
    # !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.bin -d ./extensions/controlnet/models -o ip-adapter-plus-face_sd15.pth

    # #extra
    # !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -d ./extensions/controlnet/models -o control_v11p_sd15_mlsd_fp16.safetensors
    # !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -d ./extensions/controlnet/models -o control_v11p_sd15_normalbae_fp16.safetensors
    # !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -d ./extensions/controlnet/models -o control_v11e_sd15_shuffle_fp16.safetensors

    # #t2iadapter
    # !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -d ./extensions/controlnet/models -o t2iadapter_style_sd14v1.pth
    # !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -d ./extensions/controlnet/models -o t2iadapter_color_sd14v1.pth
    # !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd15v2.pth -d ./extensions/controlnet/models -o t2iadapter_sketch_sd15v2.pth
    # !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_zoedepth_sd15v1.pth -d ./extensions/controlnet/models -o t2iadapter_zoedepth_sd15v1.pth
    
    # #etc
    # !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ioclab_sd15_recolor.safetensors -d ./extensions/controlnet/models -o ioclab_sd15_recolor.safetensors  


In [ ]:
# 다른 모델(checkpoint) 다운로드
Checkpoint_Url = 'https://civitai.com/api/download/models/94640?type=Model&format=SafeTensor&size=pruned&fp=fp16'
Checkpoint_Filename = 'majicMIX_realistic_v6.safetensors'

if 'HomePath' in locals():
    %cd {HomePath}/{Workspace}

    # 모델삭제
    !rm ./models/Stable-diffusion/*

    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{Checkpoint_Url}" -d ./models/Stable-diffusion -o {Checkpoint_Filename}


In [ ]:
# 시작 (localtunnel 터널링)

%cd {HomePath}/{Workspace}
!python launch.py --xformers --no-half-vae --enable-insecure-extension-access --theme dark --ngrok [ngrok키]

In [ ]:
# 저장

!tar czvf {HomePath}/output.tar.gz {HomePath}/{Workspace}/outputs
!rm -r {HomePath}/{Workspace}/outputs


In [ ]:
# 삭제

!rm {HomePath}/output.tar.gz

In [ ]:
# 용량 확인

!df -h {HomePath}

!du -sh {HomePath}/.[^.]*
!du -sh {HomePath}/*
